In [1]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np

In [2]:
# Import the datasets 
senseData = pd.read_csv("Lerror1.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:11:01.023211343,PresenceSensor,door_A
1,2022-09-02,2022-09-02T08:11:05.058745248,FloorSensor,sens4_A
2,2022-09-02,2022-09-02T08:11:07.680065591,FloorSensor,sens1_A
3,2022-09-02,2022-09-02T08:14:07.922706277,FloorSensor,sens1_A
4,2022-09-02,2022-09-02T08:14:11.751133401,FloorSensor,sens3_A
...,...,...,...,...
7305,2022-12-10,2022-12-10T08:36:33.935119910,FloorSensor,sens1_D
7306,2022-12-10,2022-12-10T08:37:03.935119910,FloorSensor,sens1_D
7307,2022-12-10,2022-12-10T08:37:33.935119910,FloorSensor,sens1_D
7308,2022-12-10,2022-12-10T08:38:03.935119910,FloorSensor,sens1_D


In [3]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:11:01.023211343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02T08:11:05.058745248,FloorSensor,sens4_A,A,0
2,2022-09-02,2022-09-02T08:11:07.680065591,FloorSensor,sens1_A,A,0
3,2022-09-02,2022-09-02T08:14:07.922706277,FloorSensor,sens1_A,A,0
4,2022-09-02,2022-09-02T08:14:11.751133401,FloorSensor,sens3_A,A,0
...,...,...,...,...,...,...
7305,2022-12-10,2022-12-10T08:36:33.935119910,FloorSensor,sens1_D,D,456
7306,2022-12-10,2022-12-10T08:37:03.935119910,FloorSensor,sens1_D,D,456
7307,2022-12-10,2022-12-10T08:37:33.935119910,FloorSensor,sens1_D,D,456
7308,2022-12-10,2022-12-10T08:38:03.935119910,FloorSensor,sens1_D,D,456


In [4]:
# Convert the date columns in datetime type
newdataset['Timestamp'] = pd.to_datetime(newdataset['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:11:01.023211343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02 08:11:05.058745248,FloorSensor,sens4_A,A,0
2,2022-09-02,2022-09-02 08:11:07.680065591,FloorSensor,sens1_A,A,0
3,2022-09-02,2022-09-02 08:14:07.922706277,FloorSensor,sens1_A,A,0
4,2022-09-02,2022-09-02 08:14:11.751133401,FloorSensor,sens3_A,A,0
...,...,...,...,...,...,...
7305,2022-12-10,2022-12-10 08:36:33.935119910,FloorSensor,sens1_D,D,456
7306,2022-12-10,2022-12-10 08:37:03.935119910,FloorSensor,sens1_D,D,456
7307,2022-12-10,2022-12-10 08:37:33.935119910,FloorSensor,sens1_D,D,456
7308,2022-12-10,2022-12-10 08:38:03.935119910,FloorSensor,sens1_D,D,456


In [5]:
out = pd.DataFrame(columns = ["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
i = 0
for idc, case in newdataset.groupby("Case_ID"):
    for id, window in case.groupby(pd.Grouper(freq="30s", key = "Timestamp")):
        if(window.empty):
            pass
        else:
            most_freq_label = window['Label'].value_counts().idxmax()
            dict = {"Case_ID":[],"Timestamp":[],"Type":[],"Sensor":[],"Label":[],"Activity_ID":[]}
            for a, el in window.iterrows():
                dict.get("Case_ID").append(el.get("Case_ID"))
                dict.get("Timestamp").append(el.get("Timestamp"))
                dict.get("Type").append(el.get("Type"))
                dict.get("Sensor").append(el.get("Sensor"))
                dict.get("Label").append(most_freq_label)
                dict.get("Activity_ID").append(i)
            i = i+1
            df2 = pd.DataFrame(dict)
            out = pd.concat([out, df2], ignore_index = True)
            # if(i==20):
            #     break

In [6]:
out

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:10:55.442288562,PresenceSensor,door_C,C,0
1,2022-09-02,2022-09-02 08:10:59.063608905,FloorSensor,sens4_C,C,0
2,2022-09-02,2022-09-02 08:11:01.023211343,PresenceSensor,door_A,A,1
3,2022-09-02,2022-09-02 08:11:03.306249591,FloorSensor,sens2_C,A,1
4,2022-09-02,2022-09-02 08:11:05.058745248,FloorSensor,sens4_A,A,1
...,...,...,...,...,...,...
7305,2022-12-10,2022-12-10 08:36:33.935119910,FloorSensor,sens1_D,D,4943
7306,2022-12-10,2022-12-10 08:37:03.935119910,FloorSensor,sens1_D,D,4944
7307,2022-12-10,2022-12-10 08:37:33.935119910,FloorSensor,sens1_D,D,4945
7308,2022-12-10,2022-12-10 08:38:03.935119910,FloorSensor,sens1_D,D,4946


In [7]:
# Save the file
out.to_csv("Lerror1_labeled.csv", index=False)